## INTRODUCTION

* Hi guys, I hope you are doing fine.
* I am writing this kernel from a beginner perspective. Feel free to ask anything unclear to you.
* Keep checking I will upgrading it intermittently because my examination are going on.
* Please **Upvote** . So  I keep contributing to this notebook. Thanks  
* You can write to me also  `kumarnikhil104@gmail.com`. I will respond at my earliest.

### Loading the modules/dependencies we would be needing

In [79]:
import numpy as np #linear algebra
import pandas as pd # loading the csv files and data processing
import matplotlib.pyplot as plt # visualization library
import seaborn as sns # visualization library
import plotly.plotly as py #visualization ilbrary
from plotly.offline import init_notebook_mode, iplot # plotly offline mode
init_notebook_mode(connected = True)
import plotly.graph_objs as go # plotly graphical object

import os
print(os.listdir("../input"))
import warnings
warnings.filterwarnings("ignore")
#plt.style.use('sns') # style of plots. 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',100)
%matplotlib inline

### From above output we can see we have following files in out current directory:
  * Resources.csv
  * Schools.csv
  * Donors.csv
  * Donations.csv
  * Teachers.csv
  * Projects.csv

### we are gonna load all the data and see what information they contains 

In [76]:
resources = pd.read_csv('../input/Resources.csv',error_bad_lines = False,warn_bad_lines = False,index_col = "Project ID")
schools = pd.read_csv('../input/Schools.csv',error_bad_lines = False, warn_bad_lines = False, index_col="School ID")
donors = pd.read_csv('../input/Donors.csv', low_memory= False,index_col= "Donor ID")
donations = pd.read_csv('../input/Donations.csv',index_col="Donation ID")
teachers = pd.read_csv('../input/Teachers.csv',index_col="Teacher ID")
projects =  pd.read_csv('../input/Projects.csv',error_bad_lines=False,warn_bad_lines = False ,parse_dates=["Project Posted Date","Project Fully Funded Date"], index_col="Project ID")


In [77]:
print("Shape of the Donors.csv dataframe is: ",resources.shape)
resources.head(10)

In [4]:
print("Shape of the Donors.csv dataframe is: ",schools.shape)
schools.head(10)

In [5]:
print("Shape of the Donors.csv dataframe is: ",donors.shape)
donors.head(10)

#### Donations.csv contains information about the transactions i.e the donation relevant informations 

In [6]:
print("Shape of the Donations.csv dataframe is: ",donations.shape)
donations.head(10)

In [7]:
print("Shape of the Teachers.csv dataframe is: ",teachers.shape)
teachers.head(10)

In [8]:
print("Shape of the Donors.csv dataframe is: ",projects.shape)
projects.head(5)

### Now we have seen that all the dataframes have some information regarding the project and donations. So, we are gonna merge it all into one single dataframe which would have all the information.<br>Before that we are gonna do some visualization

In [9]:
teachers.describe()

In [10]:
plt.rcParams["figure.figsize"] = [12,6]
#teachers['Teacher Prefix'].plot(kind = 'bar')
sns.countplot(x='Teacher Prefix', data=teachers);

In [11]:
plt.rcParams["figure.figsize"] = [20,6]
donors.groupby("Donor State")['Donor City'].count().plot.bar()

In [12]:
plt.rcParams["figure.figsize"] = [12,6]
sns.countplot(x='School Metro Type',data = schools)

In [13]:
plt.rcParams["figure.figsize"] = [20,6]
#schools.groupby("School State").count().plot.bar()
schools.groupby("School State")['School District'].count().plot.bar()

### Practice some visualization on othre dataframes yourself.  I would addd few more in future. 

### Now we are gonna do some anlysis of data and as what kind of data we have? Are there missing values? What is trend in the data?

- We will start with the Donations dataframe to see how much amount people donate in general.

<h3>1) Analyzing Donations Dataframe

In [52]:
# we are gonna use pandas describe() function for this purpose also but that return output in exponential form so I prefer using `dataframe_name['column_name'].min()/max()/std() ,etc 
#donations['Donation Amount'].describe()
print("Donation:      Minimum  |   Maximum  | Mean/average     |  Median |  Std")
print("Money donated:  ",donations['Donation Amount'].min(),"   |",donations['Donation Amount'].max(),"  |",donations['Donation Amount'].mean(),"|",donations['Donation Amount'].median(),"   |",donations['Donation Amount'].std())

We can see that the minimum amount of donation is '$0'. Let find out how many such donations are there? Are ther NaN values?

In [72]:
print("Shape of the donation is: ",donations.shape)
print("--"*30)
print(donations.count())
print("--"*30)
print("No.of rows in Donation Amount which are NaN:",donations.shape[0]-donations['Donation Amount'].count())
print("% .of rows in Donation Amount which are NaN:",( (donations.shape[0]-donations['Donation Amount'].count()) / donations.shape[0])*100,"%")

We can see that there are **NaN** values in the column Donation Amount.<br>
**Note:** Outliers never tell the true spread or distribution of the data because,  they are influenced by outlieres and get biased towards them. We can use median in place to get the understanding how much people donate in general.

In [ ]:
donations['Donation Amount'].plot(kind='density', subplots= False,sharex=False,figsize =(12,6));  #layout=(1, 2), sharex=False, figsize=(12, 4));

In [97]:
plt.rcParams["figure.figsize"] = [12,6]
sns.distplot(donations['Donation Amount'].dropna(axis=0));

### There is one more column regarding donations in the dataframe i.e  <i>"Donation Included Optional Donation"</i>
This is the amount which helps the organization to keep running and includes money for their proper functioning, their resources and employees salary. we would see how many people like to do it.

In [92]:
plt.rcParams["figure.figsize"] = [12,6]
data = donations['Donation Included Optional Donation'].value_counts()[:]
data.plot.bar()

We can see majority of people who decide to donate also supports the organization.
<h3>2)Analyzing Donors Dataframeaa

In [102]:
print("Size of the Donors Dataframe is: ",donors.shape)
print("--"*30)
donors.info(null_counts=True)

**We can see there are many missing values in <i>"Donor City" </i> and<i> "Donar Zip"</i>.  May be donor want to be anonymised.**<br><hr>
Let's see how many teacher are donors.

In [117]:
print(donors['Donor Is Teacher'].value_counts())
print("--"*30)
print("Percentage of teachers are not donor: %f" %(1910355/donors.shape[0] *100),"%")
sns.countplot(x=donors['Donor Is Teacher'],data = donors)

- It is visible that only 10% teachers are donors also. That raises a question<br>
**Q)** How many Donors are unique or there are some people who keeps donating?

In [124]:
print("Total no. of Donors in Donors Dataframe are:",donors.shape[0])
same_donor = donations['Donor ID'].value_counts().to_frame()
print('People Donating more than once are: ',same_donor[ same_donor['Donor ID']>1].shape[0])
print(" % of people donating again are :",  (same_donor[ same_donor['Donor ID']>1].shape[0]/same_donor.shape[0] *100) )

**Q) I'm curious to know which city peoples are most generous?** (Try plotting it by yourself, I will do it in next update. ) You will find out something interesting .

### Notebook Under constant upgrade. Keep visiting and cast an <i>Upvote</i>